<a href="https://colab.research.google.com/github/rajeevo/ML_Keras_Save_-_Load/blob/master/ML_Keras_Save_%26_Load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1.) Import Libraries

In [ ]:
!pip install -q pyyaml h5py # required to save models in HDF5 format

In [ ]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.2.0


In [ ]:
# 2.) Get an Example Dataset

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
# 3.) Define a Model

In [ ]:
# Define a simple sequential model
def create_model():
  model = tf.keras.models.Sequential([
                                      keras.layers.Dense(512, activation='relu', input_shape=(784,)),
                                      keras.layers.Dropout(0.2),
                                      keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  
  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 4.) Save Checkpoints During Training

In [ ]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images,
          train_labels,
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback]) # Pass callback to training

Epoch 1/10
25/32 [======================>.......] - ETA: 0s - loss: 1.3307 - accuracy: 0.6137
Epoch 00001: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 19ms/step - loss: 1.1906 - accuracy: 0.6540 - val_loss: 0.7213 - val_accuracy: 0.7740
Epoch 2/10
24/32 [=====================>........] - ETA: 0s - loss: 0.4440 - accuracy: 0.8776
Epoch 00002: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 13ms/step - loss: 0.4335 - accuracy: 0.8770 - val_loss: 0.5756 - val_accuracy: 0.8130
Epoch 3/10
27/32 [========================>.....] - ETA: 0s - loss: 0.2887 - accuracy: 0.9167
Epoch 00003: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 14ms/step - loss: 0.2936 - accuracy: 0.9190 - val_loss: 0.4759 - val_accuracy: 0.8480
Epoch 4/10
29/32 [==========================>...] - ETA: 0s - loss: 0.1999 - accuracy: 0.9526
Epoch 00004: saving model to training_1/cp.ckpt
32/32 [==============================] - 0

In [ ]:
ls {checkpoint_dir}

checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


In [ ]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.3894 - accuracy: 0.1030
Untrained model, accuracy: 10.30%


In [ ]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4168 - accuracy: 0.8620
Restored model, accuracy: 86.20%


In [ ]:
# Include the epoch in the file name (uses 'str.format')
checkpoint_path = "training_2/cp-{epoch:04}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    period=5)

# Create a new model instance
model = create_model()

# Save the weights using the 'checkpoint_path' format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images,
          train_labels,
          epochs=50,
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [ ]:
ls {checkpoint_dir}

checkpoint                        cp-0025.ckpt.index
cp-0000.ckpt.data-00000-of-00001  cp-0030.ckpt.data-00000-of-00001
cp-0000.ckpt.index                cp-0030.ckpt.index
cp-0005.ckpt.data-00000-of-00001  cp-0035.ckpt.data-00000-of-00001
cp-0005.ckpt.index                cp-0035.ckpt.index
cp-0010.ckpt.data-00000-of-00001  cp-0040.ckpt.data-00000-of-00001
cp-0010.ckpt.index                cp-0040.ckpt.index
cp-0015.ckpt.data-00000-of-00001  cp-0045.ckpt.data-00000-of-00001
cp-0015.ckpt.index                cp-0045.ckpt.index
cp-0020.ckpt.data-00000-of-00001  cp-0050.ckpt.data-00000-of-00001
cp-0020.ckpt.index                cp-0050.ckpt.index
cp-0025.ckpt.data-00000-of-00001


In [ ]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [ ]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4838 - accuracy: 0.8730
Restored model, accuracy: 87.30%


In [ ]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4838 - accuracy: 0.8730
Restored model, accuracy: 87.30%


In [ ]:
# 5.) Save the Entire Model

In [ ]:
# Create and train a new model instance
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 0s 6ms/step - loss: 1.2105 - accuracy: 0.6440
Epoch 2/5
32/32 [==============================] - 0s 6ms/step - loss: 0.4536 - accuracy: 0.8640
Epoch 3/5
32/32 [==============================] - 0s 6ms/step - loss: 0.2991 - accuracy: 0.9120
Epoch 4/5
32/32 [==============================] - 0s 6ms/step - loss: 0.2342 - accuracy: 0.9410
Epoch 5/5
32/32 [==============================] - 0s 6ms/step - loss: 0.1648 - accuracy: 0.9690
INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [ ]:
# my_model directory
!ls saved_model

# Contains an assets folder, saved_model.pb, and variables folder
!ls saved_model/my_model

mymodel  my_model
assets	saved_model.pb	variables


In [ ]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_11 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4418 - accuracy: 0.8590
Restored model, accuracy: 85.90%
(1000, 10)


In [ ]:
# 6.) HDF5 Format

In [ ]:
# Create and train a new model instance
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file
# The '.h5' extension indicates that the model should be saved to HDF5
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 0s 4ms/step - loss: 1.1516 - accuracy: 0.6830
Epoch 2/5
32/32 [==============================] - 0s 5ms/step - loss: 0.4477 - accuracy: 0.8760
Epoch 3/5
32/32 [==============================] - 0s 5ms/step - loss: 0.3092 - accuracy: 0.9200
Epoch 4/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2056 - accuracy: 0.9530
Epoch 5/5
32/32 [==============================] - 0s 5ms/step - loss: 0.1573 - accuracy: 0.9650


In [ ]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_12 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [ ]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 0.4296 - accuracy: 0.8680
Restored model, accuracy: 86.80%
